# ALBERT (A Lite BERT)
- BERT는 원논문에서 제시한 모델이 아직 undertrain 되어있다고 말할 수 있을 정도로 모델의 크기가 다소 커 학습 시간이나 데이터가 더 많이 요구되었다.

❗ BERT에서 다음의 사항을 개선하여 모델의 크기는 줄이고 GLUE, SQuAD, RACE Task에 대해 더 높은 성능을 얻음

**Sentence order prediction**  
- BERT에서의 Next Sentence Prediction(NSP) 대신 두 문장 간 순서를 맞추는 방식으로 학습 (Masked LM 방식은 동일하게 활용) -> NSP 보다 SQUAD, MNLI, RACE task에서 성능 향상

In [ ]:
is_next = rand() < 0.5 # whether token_b is next to token_a or not

tokens_a = self.read_tokens(self.f_pos, len_tokens, True)
seek_random_offset(self.f_neg)
#f_next = self.f_pos if is_next else self.f_neg
f_next = self.f_pos # `f_next` should be next point
tokens_b = self.read_tokens(f_next, len_tokens, False)

if tokens_a is None or tokens_b is None: # end of file
    self.f_pos.seek(0, 0) # reset file pointer
    return

# SOP, sentence-order prediction
instance = (is_next, tokens_a, tokens_b) if is_next \
                    else (is_next, tokens_b, tokens_a)

  **Cross-layer parameter sharing**   
- Transformer의 각 Layer 간 같은 Parameter를 공유하여 사용 -> 모델 크기 줄임

In [ ]:
class Transformer(nn.Module):
    """ Transformer with Self-Attentive Blocks"""
    def __init__(self, cfg):
        super().__init__()
        self.embed = Embeddings(cfg)
        # Original BERT not used parameter-sharing strategies
        # self.blocks = nn.ModuleList([Block(cfg) for _ in range(cfg.n_layers)])

        # To used parameter-sharing strategies
        self.n_layers = cfg.n_layers
        self.attn = MultiHeadedSelfAttention(cfg)
        self.proj = nn.Linear(cfg.hidden, cfg.hidden)
        self.norm1 = LayerNorm(cfg)
        self.pwff = PositionWiseFeedForward(cfg)
        self.norm2 = LayerNorm(cfg)
        # self.drop = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, x, seg, mask):
        h = self.embed(x, seg)

        for _ in range(self.n_layers):
            # h = block(h, mask)
            h = self.attn(h, mask)
            h = self.norm1(h + self.proj(h))
            h = self.norm2(h + self.pwff(h))

        return h

**Factorized embedding parameterization**   
- Input Layer의 Parameter 수를 줄임 -> 모델 크기 줄임  

In [ ]:
class Embeddings(nn.Module):
    "The embedding module from word, position and token_type embeddings."
    def __init__(self, cfg):
        super().__init__()
        # Original BERT Embedding (token embedding)
        # self.tok_embed = nn.Embedding(cfg.vocab_size, cfg.hidden) 
        
        # factorized embedding
        self.tok_embed1 = nn.Embedding(cfg.vocab_size, cfg.embedding)
        self.tok_embed2 = nn.Linear(cfg.embedding, cfg.hidden)

        # position embedding
        self.pos_embed = nn.Embedding(cfg.max_len, cfg.hidden) 
        # segment(token type) embedding
        self.seg_embed = nn.Embedding(cfg.n_segments, cfg.hidden)

---

# RoBERTa

### 기존 BERT의 경우 
- Model
    - transformer encoder(with Multi-head Attention, FFN) x12
- Learning WITH
    - Masked Language Modeling(MLM)
    - Next Sentence Prediction(NSP)
- Optimization 
    - Adam Optimizer, $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\epsilon=1e-6$, $L2\;weight\;decay=0.01$
    - learning rate=1e-4, 10,000 steps warmup (10000스텝까지 1e-4로 linear warmup 이후 다시 감소)
    - GELU activation, 
    - 0.1 dropout on all layers
    - batch size 256 sequences (256 sequences * 512 tokens = 128,000 tokens/batch)
    - 1,000,000 steps (128,000 * 1,000,000 / 3.3B word corpus = 40epochs)
- character 단위 BPE tokenizer

### 반면 RoBERTa의 경우
**최적의 하이퍼파라미터로 학습**
- 더 많은 data와 더 큰 batch size(약 32배) (계산 비용은 그대로 유지)
- Peak learning rate/warm up step .. batch size에 따라 각각 튜닝
- Large batch size에서 더 안정적이도록 Adam optimizer에서 $\beta_2 = 0.98$로 설정
- input으로 들어가는 sequence의 length 길이를 항상 full(512)로 설정
- Dynamic Masking - MLM 수행 시 매 epoch마다 다른 masking을 활용
    + data를 10개 복제하여 각 sequence가 40 epoch에 걸쳐 10가지 방법으로 masking 되도록 처리
    + 학습 중 동일한 mask는 4번만 보게 된다. 
- NSP 제거
    + NSP는 사실 문장이 붙어있어서 성능이 좋았던거지, 다음 문장 예측으로 학습해서 성능이 좋았던게 아니다.
    + 그냥 모든 문장을 붙여서 최대한 길이를 길게(512)해서 넣으면 성능 더 좋았음
- byte단위 BPE tokenizer

---

# Cross-Lingual Language Model
### 데이터가 풍부하지 않은 언어를 위해 transfer learning을 진행하는 cross-lingual transfer learning 분야의 연구
### 물론 이렇게 학습하는 데에는 다른 이유도 있을 것이다. 번역이 좀 더 쉬워진다든지 등..

# XLM

- Unsupervised, relies on monolingual data
- Supervised, relies on parallel data.

**The model uses the same shared vocabulary for all the languages.**
- 여러 언어가 하나의 임베딩 공간을 공유하고 어떤 언어로 쓰인 문장도 해당 임베딩 공간으로 인코딩 되도록 하는 유니버셜 인코더를 만들고자 하였다.
=> 비슷하게 알파벳을 쓴다던가 하는 언어들은 embedding space에서 잘 매칭됨
- BPE 활용 .. BPE를 활용하면 같은 글자(ex. 알파벳) 쓰는 언어들은 토큰 매칭이 가능하다는 장점
- 데이터가 부족한 언어에 transfer learning이 가능
  
![XLM](https://miro.medium.com/max/1000/1*EcaP5C5UhcCqkeNqvYscTA.png)  
  
- BERT 구조
- **CLM** 일반적인 transformer decoder를 이용한 LM
- **MLM** default BERT가 input으로 sequence 2개(pair of sentences)를 활용하는 반면, XLM은 길이만 256이면 그냥 문장 몇개든 사용
- **TLM** 위와 같이 서로 다른 두 parallel 문장을 두고 mask함. **모든 토큰은 각 마스크 예측에 관여**
- Language embedding token 활용
- 일반 BERT에서 pretrained 된 token을 미리 가져온 후 그것으로 학습을 시작했더니 성능이 더 좋았다는 결과

---

# XLM-RoBERTa(XLM-R)  >> mBERT
## Unsupervised Cross-lingual Representation Learning at Scale
[원 논문](https://arxiv.org/pdf/1911.02116.pdf)
- Building on the cross-lingual approach that we used with XLM and RoBERTa
- It is trained on 2.5T of data across 100 languages data filtered from Common Crawl. 
- XLM-R achieves state-of-the-arts results on multiple cross lingual benchmarks.
- 기존 multi-lingual model인 mBERT에 비해 **데이터양이 적은 언어에서 정확도 23% 향상**

<br />

- 일단 이전 모델들이 학습할 때 사용한 데이터의 양이 적었음을 지적 => 사용하는 데이터를 엄청나게 늘림
- 단일 언어에 대한 성능에서도 경쟁력 있음(RoBERTa와 견줄만 함)
- Subword tokenization .. SentencePiece with unigram
- 250K vocab size
- 주로 모델 자체의 구조보다는 어떤 데이터로 학습했으며 성능은 어땠는지를 설명하고 있음.
    + monolingual의 성능에 꿇리지 않고, 오히려 cross-lingual(XLM 기반) 모델이 성능을 앞지를 때도 있음을 보이며 가능성을 제시
- RoBERTa 백본

- multi-lingual language modeling을 위해 monolingual data로 학습
- 모든 언어에서 text stream을 추출하여 mask token 예측 학습

### Curse of Multi-linguality
![curse_of_multilinguality](https://miro.medium.com/max/700/1*xQZN1oci9xAKDdA9gBNXGA.png)
- While we improve the performance for low-resource languages, this ‘dilution’ degrades the overall performance on the downstream tasks. This dilution has a trade-off with the model capacity.
- multilingual 모델 설계의 장점은 데이터가 적은 언어에서도 보다 좋은 성능을 낼 수 있게 된다는 것인데, 이를 위해 언어를 추가하면(dilution) 반대로 overall performance는 떨어지는 tradeoff가 있음.
- 하지만 위 그림에서 보이듯이 처음에 성능 증가하다가 하락. 결국 모델 크기 자체를 크게 늘리면 이것도 해결할 수 있음을 제시
- 실제로 parameter를 조정하고, low-resource language를 upsampling하고, large shared vocabulary를 생성, capacity 자체를 향상시킴